In [4]:
import numpy as np

import matplotlib.pyplot as plt
from ipywidgets import interactive, interact
from matplotlib import animation, rc
import ipywidgets as widgets
%matplotlib widget
from scipy import fftpack as fft
from scipy import signal as sig
import pandas as pd
from IPython.display import display, HTML 

#Constants
fs = 1000000 # Samples/seconds

In [5]:
plt.close('all')

In [3]:
#Import Data
raw0 = np.fromfile(open("open_press.smpl"), dtype=np.complex64)

#raw0 with respect to time
Traw = np.linspace(0.0, raw0.size/fs, raw0.size)

In [6]:
def calcFFT(data, fs):
    yf = fft.fft(data)
    yf = fft.fftshift(yf)
    xf = fft.fftfreq(data.size, 1/fs)
    xf = fft.fftshift(xf)
    return [xf, yf]

In [23]:
#Lets view raw signal and its FT
plt.close("Raw Data")
plt.figure("Raw Data")
plt.plot(Traw, raw0.real, Traw, raw0.imag)
plt.show()

rawFFT = calcFFT(raw0, fs)

plt.close("FFT of Raw Data")
plt.figure("FFT of Raw Data")
plt.plot(rawFFT[0], rawFFT[1].real, rawFFT[0], rawFFT[1].imag)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
#TODO: To optimize integrate this into find packets method

#The FMCOMMS3 uses an automatic gain control. Haven't figured out how to config manual gain control correctly so I need to remove pre-gain correction.
#Note: set gain to ~45dB and configure gr-iio to manual gain mode in GRC TODO: nake this work.

sos = sig.butter(15, 100, 'lp', fs=fs, output='sos')
rawf = sig.sosfilt(sos, raw0)
gain_set =abs(rawf)>.005  
gain_set_loc = np.argmax(gain_set)
raw = raw0[gain_set_loc:]

#Show successful isolation. New dataset is in orange
plt.close("Remove FMCOMMS Gain Instability")
plt.figure("Remove FMCOMMS Gain Instability")
plt.plot(Traw, raw0.real, Traw[gain_set_loc:], raw.real)
plt.show()

T = np.linspace(0.0, raw.size/fs, raw.size)
#Plot with new time reference
plt.close("Received Broadcast Plotted Over Time")
plt.figure("Received Broadcast Plotted Over Time")
plt.plot(T, raw.real, T, raw.imag)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
#Now that we can hypothosize that the remote sends a same/similar message the longer the button is pressed, we can 
#isolatethe message based on the first occurence 
#Maybe optimal method is to use derivative like what is used to isolate bits

def isolatePacket(raw_data):
    SMPL_BUFF = 2000 # buffer to 
    PCKT_SIZE_EST = 50000
    tx =abs(raw_data)>.05  
    msg_loc = np.where(tx == True)[0] #locatoins where value non-zero
    start_tx = msg_loc[0] - SMPL_BUFF #I know this is dangerous but I'm a badass
    
    #find end of packet
    diff_msg_loc = np.roll(msg_loc, 1)
    diff_msg_loc[0] = msg_loc[0]
    temp = np.where((msg_loc - diff_msg_loc) > PCKT_SIZE_EST)[0]
    end_tx = msg_loc[temp[0]] - PCKT_SIZE_EST + SMPL_BUFF
    
    return [start_tx, end_tx]


pkt_size = isolatePacket(raw)
raw_pkt = raw[pkt_size[0]:pkt_size[1]]
x_raw_pkt = T[pkt_size[0]:pkt_size[1]] #For plotting in time
#x_raw_pkt = range(raw_pkt.real.size)

print("PKT Start in time:", T[pkt_size[0]+2000])
print("PKT End in time is about:", T[pkt_size[1]-2000])

plt.close("Isolated Packet")
plt.figure("Isolated Packet")
plt.plot(x_raw_pkt, raw_pkt.real, x_raw_pkt, raw_pkt.imag)
plt.show()

#Find dominant frequency/ carrier freq? Not sure if that's the right term
raw_pktFFT = calcFFT(raw_pkt, fs)
temp = np.max(np.abs(raw_pktFFT[1]))
F0_index = np.where(np.abs(raw_pktFFT[1]) == temp)
F0 = raw_pktFFT[0][(F0_index[0])][0]
print("Dominant Freq is:", F0)

PKT Start in time: 0.044438011136349743
PKT End in time is about: 0.0941330235901258


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Dominant Freq is: 5791.973181860509


## Initial Analysis of packets
### Packet Makeup
Each packet seems to consist of a preamble followed by a message body. In phase (I) and quadrature (Q) seems to be out of phase by a consistent amount. Looks like 90 degree phase difference. 
#### Preamble Data
The preamble consists of 6 pulses.

| Pulse # | Periods | Positive Phase Angle (<180deg) | Time difference b/t I and Q | Pulse Frequency |
| ------- | ------- | ------------------------------ | --------------------------- | --------------- |
| 1       | 1.5     | +                              | .0000376                    |
| 2       | 3       | +                              | .0000439                    |
| 3       | 2.5     | +                              | .0000461                    |
| 4       | 2.5     | -                              | .0000438                    | 5792 |
| 5       | 3       | -                              | .0000439                    | 5792 |
| 6       | 1.5     | +                              | .0000427                    | 5733 |


Average time difference in phase b/t I and Q is 440.8 (excluding Pulse 1 outlier). Frequency of pulses appears to be ~5792 Hz. I stopped measuring because it's smarter to just look at where the max FFT value occurs. Max off FFT put dominant freq at 5.791 kHz. This is in line with manual calculations. The phase can difference b/t I and Q data is estimated to be 91.9 degrees. Basically 90 degrees. I guess this should've been assumed because of the definition of quadrature. 

If the protocol is anything like CAN, SPI, I2C, I assume that the preamble contains some sort of address.

#### Body Data
Body conisists of 58 pulses. Each pulse seems to vary in number of periods. In addition, the length of time b/t each pulse seems to vary as well. Table includes first 6 pulses. I don't feel like manually categorizing them all.

| Pulse # | Periods | Positive Phase Angle (<180deg) |
| ------- | ------- | ------------------------------ |
| 1       | 1.5     | -                              |
| 2       | 3       | -                              |
| 3       | 2.5     | -                              |
| 4       | 3       | +                              |
| 5       | 3       | -                              |
| 6       | 1.5     | +                              |

#### Hypotheses
* BPSK: Bits values are encoded into the phase of the incoming signal
    * The phase differences allude to a BPSK scheme
* ASK (Amplitude Shift-Keying): Bit values are determined by time, similar to electrical signals. 
    * The periodic differences allude to time encoding method
    * TBH I think that this is the most likely cast given that a car key is a low cost device. It seems cheapest to decode.
    * ON/OFF keying
    * this also accounts for the differences in phases. 

### Decoding Courses of Action
How do I prove / disprove each hypothesis?
#### ASK
Since this is the most likely COA, it will be the first I explore
1. Demod to squarish wave and determine duration of each pulse/bit
    * LPF/Integrate
    * Mix w/ baseband freq - this poses the problem of needing to synch phase. However the pro is that the bit times would be more accurate.
1. Convert pulses to bits
    * Integrate and Dump
    * LPF the entire thing and then use bit timing
    * LPF and identify time intervals.
    * I guess both ways mentioned above are essentially the same. LPF and integrate are basically the same thing I'm pretty sure. It just comes down to which is less computationally intensive. (Kids this is why you don't join the army. It makes you stupid and forget things you learnt sophomore year of college.)
1. Compare periods/time of each pulse accross multiple packets of the same button press
1. Compare periods/time of each pulse accross multiple packets with different button presses

#### BPSK
1. Determine phase of each pulse
1. Compare phase accross multiple packets of the same button press
1. Compare phase accross multiple packets with different button presses



In [7]:
#Function implementation of above method to remove the noisy start before AGC corrects
def AGC_Correct(raw, fs):
    sos = sig.butter(15, 200, 'lp', fs=fs, output='sos')
    rawf = sig.sosfilt(sos, raw)
    rawf = rawf[10000:] #Discard 1st 10k samples b/c LPF needs to become stable
    gain_set =abs(rawf) < .025  
    gain_set_loc = np.argmax(gain_set)

    return raw[gain_set_loc+10000:]

In [8]:

#If the key uses on/off keying then we don't care about complex values. I used the complex to Mag block in GRC to get a float version of the key data
rawfl = np.fromfile(open("open_press_mag.smpl"), dtype=np.float32)


#Isolate part where FMCOMMS gain is all F'ed up
rawflt = AGC_Correct(rawfl, fs) 
Tflt = np.linspace(0.0, rawflt.size/fs, rawflt.size) #rawflt with respect to time

rawflt /= rawflt.max() #Normalize

plt.close("Real Value Transmission")
plt.figure("Real Value Transmission")
plt.plot(Tflt, rawflt)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

It's pretty clear from the magnitude plot that this is ON/OFF keying. Wow I feel stupid for spending this much time analyzing the complex data and thinking it might be BPSK. Lesson learned is to start by looking at the magnitude/float data before looking at the complex data. 

Next step is to find the baud rate. I presume the easiest way to do this is to look at the FFT and find the freq w/ highest gain.

In [9]:
plt.close('FFT of Real Car Key Transmission')
rawFFT = calcFFT(rawflt, fs)
#Zero out DC Component
dc_index = np.int(np.size(rawFFT[0])/2)
rawFFT[1][dc_index] = 0 # Zero out DC component
baudrate_mag = np.argmax(abs(rawFFT[1]))
baudrate = rawFFT[0][baudrate_mag]
print("Dominant Frequency: ", baudrate)

plt.figure("FFT of Real Car Key Transmission")
plt.plot(rawFFT[0], rawFFT[1].real)
plt.show()

Dominant Frequency:  -9.99528370687313


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

My previous assumption that the dominant frequency would be the baud rate is incorrect. 10Hz is too slow to be the baud rate. Instead 10Hz probably refers to the packet transmission rate. When the key button is pressed, the key transmits the same packet at 10Hz. This was confirmed visually

I'll attempt to calculate the baud rate by taking the differential of the received real signal. I'll do this by subtracting the original signal from the original shifted 2 samples. The instantaneous changes that occur when a bit changes from 0 to 1 should show up as spikes. I'm shifting by 2 samples because after graphical analysis of the bit rise time, it looks like it takes 2 samples for a bit to go from 0 to 1.

In [10]:
diff = np.roll(rawflt, -2)    #diff is a copy of rawflt shifted 2 to the left
diff = rawflt - diff

plt.close("Differential of Real Signal")
plt.figure("Differential of Real Signal")
plt.plot(Tflt, rawflt, Tflt, abs(diff)>.5)
plt.show()

temp = np.where(abs(diff)>.5)
#bit_times = Tflt[temp] - np.roll(Tflt[temp], -1)
bit_times = abs(temp[0] - np.roll(temp[0], -1))
bit_times[0] = 0
#np.savetxt("bit_transitions.csv", np.column_stack((temp[0], bit_times)), delimiter=',')


plt.close("Dist of Bit Times in Samples")
plt.figure("Dist of Bit Times in Samples")
bins= plt.hist(bit_times, bins=100, range=(20, 600))
plt.show()

bins = np.array([bins[0],bins[1]])
bins = [bins[0], bins[1][0:-1]]
np.savetxt("bit_hist.csv", np.column_stack((bins[0], bins[1])), delimiter=',')
bin_table = pd.DataFrame(bins)
display(bin_table)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20.0,25.8,31.6,37.4,43.2,49.0,54.8,60.6,66.4,72.2,...,542.0,547.8,553.6,559.4,565.2,571.0,576.8,582.6,588.4,594.2


I had originally assumed that each bit value was determined soley if the signal was HIGH or LOW at a specified time (kind of like I2C where the bit value of the data signal is recorded when the clock signal transitions from LOW to HIGH). After examining the histogram of bit times and looking again at the real signal, it looks like manchester encoding is used. The majority of bit transitions took ~250 samples and ~500 samples. This lines up with the enoding scheme. The signal will take twice as long to transition from HIGH to LOW or vice versa when the bit value changes than it would take if the bit value stayed the same. This hypothesis also makes sense b/c is allows the receiver to self-synchronize quite easily. The wikipedia article also mentioned that manchester encoding is commonly used in consumer IR protocols. 

In [19]:
#LPF fx. 5th order BW filter
#f_cutoff is the cutoff freq for the lpf
def lpf(data, fs, f_cutoff):
    sos = sig.butter(15, f_cutoff, 'lp', fs=fs, output='sos')
    filtered = sig.sosfilt(sos, raw0)

    return filtered

In [18]:
#Understanding Quadrature Demod

freq = 10
t = np.arange(0, 4, .01)
x = np.cos(freq*t + 1*np.pi/4)
x2 =np.cos(freq*t)
x3 = np.cos(freq*t + np.pi/4)
x4 = np.cos(freq*t + np.pi/2)
x5 = np.cos(freq*t + 3*np.pi/4)
plt.figure()
#Plot original signal
plt.subplot(411)
plt.plot(t, x2, t, x3, t, x4, t, x5)

#Implement Quadrature demod
comp_data = x*((np.cos(freq*t)) - 1j*np.sin(freq*t))
plt.subplot(412)
plt.plot(t, comp_data.real, t, comp_data.imag)

#Plot Symbol
plt.subplot(413)
plt.xlim(-1,1)
plt.ylim(-1,1)
#symb = comp_data
symb = np.sum(comp_data)
symb = symb / abs(symb)
plt.scatter(symb.real, symb.imag)

plt.subplot(414)
plt.plot(t, comp_data.real+comp_data.imag, t, x)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …